In [6]:
from web3 import Web3
from abi import UNISWAP_FACTORY_ABI, UNISWAP_PAIR_ABI, ERC20_ABI
from itertools import combinations


In [45]:

# Configuração da rede Polygon
# polygon_rpc_url = "https://polygon-rpc.com"
# web3 = Web3(Web3.HTTPProvider(polygon_rpc_url))

polygon_rpc_url = "https://polygon-mainnet.infura.io/v3/079d35a3c196469191c9accc1d68d25b"
web3 = Web3(Web3.HTTPProvider(polygon_rpc_url))

# Verificando conexão
if not web3.provider.is_connected():
    raise ConnectionError("Falha ao conectar à rede Polygon")

print("Conectado à rede Polygon")

# Endereços das fábricas das DEXs
DEX_FACTORIES = {
    "QuickSwap": web3.to_checksum_address("0x5757371414417b8c6caad45baef941abc7d3ab32"),
    "SushiSwap": web3.to_checksum_address("0xc35dadb65012ec5796536bd9864ed8773abc74c4")
}

# Exemplo de uso do endereço checksum
print("Endereços no formato checksum:")
for name, address in DEX_FACTORIES.items():
    print(f"{name}: {address}")



Conectado à rede Polygon
Endereços no formato checksum:
QuickSwap: 0x5757371414417b8C6CAad45bAeF941aBc7d3Ab32
SushiSwap: 0xc35DADB65012eC5796536bD9864eD8773aBc74C4


In [46]:
def get_decimals(token_address):
    token_contract = web3.eth.contract(address=token_address, abi=ERC20_ABI)
    return token_contract.functions.decimals().call()

# Função para obter reservas de um par de tokens em uma DEX
def get_token_reserves(factory_address, token_a, token_b):
    factory = web3.eth.contract(address=factory_address, abi=UNISWAP_FACTORY_ABI)
    pair_address = factory.functions.getPair(token_a, token_b).call()
    
    if pair_address == "0x0000000000000000000000000000000000000000":
        return None

    pair = web3.eth.contract(address=pair_address, abi=UNISWAP_PAIR_ABI)
    reserves = pair.functions.getReserves().call()

    # Identificar a ordem dos tokens no par
    token0 = pair.functions.token0().call()
    token1 = pair.functions.token1().call()

    # Obter os decimais corretos para cada token
    decimals_token0 = get_decimals(token0)
    decimals_token1 = get_decimals(token1)

    # Retornar as reservas na ordem correta
    if token0.lower() == token_a.lower():
        return (reserves[0] / (10 ** decimals_token0), reserves[1] / (10 ** decimals_token1))
    else:
        return (reserves[1] / (10 ** decimals_token1), reserves[0] / (10 ** decimals_token0))


def simulate_swap(investment, reserve_in, reserve_out):
    """
    Simula um swap baseado no modelo x * y = k.
    Retorna a quantidade do token recebido.
    """
    if reserve_in == 0 or reserve_out == 0 or investment == 0:
        return 0
    return (reserve_out * investment) / (reserve_in + investment)

def calculate_arbitrage(factory_a, factory_b, token_a, token_b, investment, symbol_a, symbol_b):
    reserves_a = get_token_reserves(factory_a, token_a, token_b)
    reserves_b = get_token_reserves(factory_b, token_a, token_b)

    if not reserves_a or not reserves_b:
        return None

    # Preços nas duas DEXs
    price_a = reserves_a[0] / reserves_a[1]
    price_b = reserves_b[0] / reserves_b[1]

    # Simulação detalhada de swaps
    # Etapa 1: QuickSwap -> SushiSwap
    received_token_b_1 = simulate_swap(investment, reserves_a[1], reserves_a[0])  # TOKEN_B -> TOKEN_A
    received_token_a_2 = simulate_swap(received_token_b_1, reserves_b[0], reserves_b[1])  # TOKEN_A -> TOKEN_B

    # Etapa 2: SushiSwap -> QuickSwap
    received_token_b_2 = simulate_swap(investment, reserves_b[1], reserves_b[0])  # TOKEN_B -> TOKEN_A
    received_token_a_1 = simulate_swap(received_token_b_2, reserves_a[0], reserves_a[1])  # TOKEN_A -> TOKEN_B

    # Resultados formatados
    details = [
        ["Step", "Action", "DEX", "Investment", "Received"],
        ["1", "Swap", "QuickSwap -> SushiSwap", f"{investment:.2f} {symbol_b}", f"{received_token_b_1:.6f} {symbol_a}"],
        ["2", "Swap", "SushiSwap -> QuickSwap", f"{received_token_b_1:.6f} {symbol_a}", f"{received_token_a_2:.2f} {symbol_b}"],
        ["3", "Swap", "SushiSwap -> QuickSwap", f"{investment:.2f} {symbol_b}", f"{received_token_b_2:.6f} {symbol_a}"],
        ["4", "Swap", "QuickSwap -> SushiSwap", f"{received_token_b_2:.6f} {symbol_a}", f"{received_token_a_1:.2f} {symbol_b}"]
    ]

    # Determinando o lucro
    profit_a_to_b = received_token_a_2 - investment  # Lucro no ciclo QuickSwap -> SushiSwap
    profit_b_to_a = received_token_a_1 - investment  # Lucro no ciclo SushiSwap -> QuickSwap

    # Determinando a melhor rota
    if profit_a_to_b > profit_b_to_a and profit_a_to_b > 0:
        return details, (f"Arbitragem: Compre na DEX A (QuickSwap) e venda na DEX B (SushiSwap). "
                         f"Lucro esperado: {profit_a_to_b:.6f} USDC.")
    elif profit_b_to_a > profit_a_to_b and profit_b_to_a > 0:
        return details, (f"Arbitragem: Compre na DEX B (SushiSwap) e venda na DEX A (QuickSwap). "
                         f"Lucro esperado: {profit_b_to_a:.6f} USDC.")
    else:
        return details, "Sem oportunidade de arbitragem lucrativa."



In [47]:

# # Configuração dos tokens para arbitragem
# TOKEN_A = web3.to_checksum_address("0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619")  # WETH
# TOKEN_B = web3.to_checksum_address("0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174")  # USDC

# # Verificação de arbitragem
# investment = 1000  # Valor investido em USDC
# for dex1, factory1 in DEX_FACTORIES.items():
#     for dex2, factory2 in DEX_FACTORIES.items():
#         if dex1 != dex2:
#             details, result = calculate_arbitrage(factory1, factory2, TOKEN_A, TOKEN_B, investment)
#             if details:
#                 print("Detalhes da Simulação:")
#                 for row in details:
#                     print(" | ".join(row))
#             print(f"\n{result}\n")

In [48]:
from itertools import combinations

TOKENS = [
    {"symbol": "WETH", "address": "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619"},  # WETH
    {"symbol": "USDC", "address": "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"},  # USDC
    {"symbol": "DAI", "address": "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063"},   # DAI
    {"symbol": "WBTC", "address": "0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6"},  # WBTC
    {"symbol": "SUSHI", "address": "0x0b3f868e0be5597d5db7feb59e1cadbb0fdda50a"},  # SUSHI
    {"symbol": "LINK", "address": "0x53e0bca35ec356bd5dddfebbd1fc0fd03fabad39"},  # LINK
    {"symbol": "UNI", "address": "0xb33eaad8d922b1083446dc23f610c2567fb5180f"},  # UNI
    {"symbol": "AVEE", "address": "0xD6DF932A45C0f255f85145f286eA0b292B21C90B"},  # AVEE
]

# Gerar pares de tokens com USDC
usdc_address = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
token_pairs = [(token, usdc) if token["address"] != usdc_address else (usdc, token) 
              for token in TOKENS if token["address"] != usdc_address
              for usdc in TOKENS if usdc["address"] == usdc_address]

# Imprimir os pares
for pair in token_pairs:
    print(f"Pares: {pair[0]['symbol']}/{pair[1]['symbol']}")

Pares: WETH/USDC
Pares: DAI/USDC
Pares: WBTC/USDC
Pares: SUSHI/USDC
Pares: LINK/USDC
Pares: UNI/USDC
Pares: AVEE/USDC


In [49]:

investment = 1000  # Valor fixo de investimento

for token_a, token_b in token_pairs:

    token_a_address = web3.to_checksum_address(token_a["address"])
    token_b_address = web3.to_checksum_address(token_b["address"])

    for dex1, factory1 in DEX_FACTORIES.items():
    # for dex2, factory2 in DEX_FACTORIES.items():
        if dex1 != dex2:
            details, result = calculate_arbitrage(factory1, factory2, token_a_address, token_b_address, investment, token_a['symbol'], token_b['symbol'])
            print(f"Pares: {token_a['symbol']}/{token_b['symbol']}")
            if details:
                print("Detalhes da Simulação:")
                for row in details:
                    print(" | ".join(row))
            print(f"\n{result}\n")

Pares: WETH/USDC
Detalhes da Simulação:
Step | Action | DEX | Investment | Received
1 | Swap | QuickSwap -> SushiSwap | 1000.00 USDC | 0.312221 WETH
2 | Swap | SushiSwap -> QuickSwap | 0.312221 WETH | 995.61 USDC
3 | Swap | SushiSwap -> QuickSwap | 1000.00 USDC | 0.311395 WETH
4 | Swap | QuickSwap -> SushiSwap | 0.311395 WETH | 995.73 USDC

Sem oportunidade de arbitragem lucrativa.

Pares: DAI/USDC
Detalhes da Simulação:
Step | Action | DEX | Investment | Received
1 | Swap | QuickSwap -> SushiSwap | 1000.00 USDC | 995.336828 DAI
2 | Swap | SushiSwap -> QuickSwap | 995.336828 DAI | 984.26 USDC
3 | Swap | SushiSwap -> QuickSwap | 1000.00 USDC | 991.125633 DAI
4 | Swap | QuickSwap -> SushiSwap | 991.125633 DAI | 983.03 USDC

Sem oportunidade de arbitragem lucrativa.

Pares: WBTC/USDC
Detalhes da Simulação:
Step | Action | DEX | Investment | Received
1 | Swap | QuickSwap -> SushiSwap | 1000.00 USDC | 0.010395 WBTC
2 | Swap | SushiSwap -> QuickSwap | 0.010395 WBTC | 197.22 USDC
3 | Swap | S

In [50]:
# Função para obter reservas de um par de tokens em uma DEX
def get_token_reserves(factory_address, token_a, token_b):
    factory = web3.eth.contract(address=factory_address, abi=UNISWAP_FACTORY_ABI)
    pair_address = factory.functions.getPair(token_a, token_b).call()
    
    if pair_address == "0x0000000000000000000000000000000000000000":
        return None

    pair = web3.eth.contract(address=pair_address, abi=UNISWAP_PAIR_ABI)
    reserves = pair.functions.getReserves().call()

    # Identificar a ordem dos tokens no par
    token0 = pair.functions.token0().call()
    token1 = pair.functions.token1().call()

    # Obter os decimais corretos para cada token
    decimals_token0 = get_decimals(token0)
    decimals_token1 = get_decimals(token1)

    # Retornar as reservas na ordem correta
    if token0.lower() == token_a.lower():
        return (reserves[0] / (10 ** decimals_token0), reserves[1] / (10 ** decimals_token1))
    else:
        return (reserves[1] / (10 ** decimals_token1), reserves[0] / (10 ** decimals_token0))


def simulate_swap(investment, reserve_in, reserve_out):
    """
    Simula um swap baseado no modelo x * y = k.
    Retorna a quantidade do token recebido.
    """
    if reserve_in == 0 or reserve_out == 0 or investment == 0:
        return 0
    return (reserve_out * investment) / (reserve_in + investment)

def calculate_arbitrage(factory_a, factory_b, token_a, token_b, investment, symbol_a, symbol_b):
    reserves_a = get_token_reserves(factory_a, token_a, token_b)
    reserves_b = get_token_reserves(factory_b, token_a, token_b)


    print(f"Reservas DEX1: {reserves_a}")
    print(f"Reservas DEX2: {reserves_b}")

    if not reserves_a or not reserves_b:
        return None

    # Preços nas duas DEXs
    price_a = reserves_a[0] / reserves_a[1]
    price_b = reserves_b[0] / reserves_b[1]

    # Simulação detalhada de swaps
    # Etapa 1: QuickSwap -> SushiSwap
    received_token_b_1 = simulate_swap(investment, reserves_a[1], reserves_a[0])  # TOKEN_B -> TOKEN_A
    return received_token_b_1


factory1 = web3.to_checksum_address("0x5757371414417b8c6caad45baef941abc7d3ab32")
factory2 = web3.to_checksum_address("0xc35dadb65012ec5796536bd9864ed8773abc74c4")
avee = '0xD6DF932A45C0f255f85145f286eA0b292B21C90B'
usdc = '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'

res = calculate_arbitrage(factory1, factory2, avee, usdc, investment, 'usdc', 'avee')
res

Reservas DEX1: (4.639639104698382, 1352.335537)
Reservas DEX2: (0.6551469038764575, 190.256333)


1.9723542971320518